In [17]:
from kubernetes import client, config
import uuid

# 加载认证文件
config.kube_config.load_kube_config("config.yml")
#获取CoreV1Api、BatchV1Api、AppsV1Api、CustomObjectsApi
core_v1 = client.CoreV1Api()  #用于操作核心v1 API组中的资源，如Pod、Service、Node、Namespace等
batch_v1 = client.BatchV1Api()  #用于操作Job等资源
apps_v1 = client.AppsV1Api()  #用于操作Deployment、DaemonSet、ReplicaSet和StatefulSet
custom_api = client.CustomObjectsApi()  #用于操作CRD

In [12]:
# 创建pod
nginx_pod = {
    'apiVersion': 'v1',
    'kind': 'Pod',
    'metadata': {
        'name': 'nginx-{}'.format(uuid.uuid4()),
        'labels': {
            "app": "nginx"
        }
    },
    'spec': {
        'containers': [
            {
                'name': 'my-container',
                'image': 'nginx:1.7.9',
                "ports": [
                    {
                        "containerPort": 80
                    }
                ]
            }
        ],
        #指定调度节点
        'nodeSelector': {
            'kubernetes.io/hostname': 'bigdata4'
        }
    }
}
"""
# 标记node
kubectl label nodes <node-name> <label-key>=<label-value>
# 展现node的label
kubectl get nodes --show-labels
# 描述node结构
kubectl describe node "nodename"
"""
core_v1.create_namespaced_pod(body=nginx_pod, namespace="default")

{'api_version': 'v1',
 'kind': 'Pod',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2023, 4, 11, 7, 34, 57, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'app': 'nginx'},
              'managed_fields': None,
              'name': 'nginx-707a92bf-ee28-4961-9a51-e580525caf87',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '41428',
              'self_link': '/api/v1/namespaces/default/pods/nginx-707a92bf-ee28-4961-9a51-e580525caf87',
              'uid': 'd9506caf-6d82-462f-8a72-874ede31186f'},
 'spec': {'active_deadline_seconds': None,
          'affinity': None,
          'automount_service_account_token': None,
          'containers': [{'args': None,
                          'command

In [13]:
# 删除pod
core_v1.delete_namespaced_pod("nginx-707a92bf-ee28-4961-9a51-e580525caf87", namespace="default")

{'api_version': 'v1',
 'kind': 'Pod',
 'metadata': {'annotations': {'cni.projectcalico.org/podIP': '10.244.191.200/32',
                              'cni.projectcalico.org/podIPs': '10.244.191.200/32'},
              'creation_timestamp': datetime.datetime(2023, 4, 11, 7, 34, 57, tzinfo=tzutc()),
              'deletion_grace_period_seconds': 30,
              'deletion_timestamp': datetime.datetime(2023, 4, 11, 7, 39, 35, tzinfo=tzutc()),
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': {'app': 'nginx'},
              'managed_fields': None,
              'name': 'nginx-707a92bf-ee28-4961-9a51-e580525caf87',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '42062',
              'self_link': '/api/v1/namespaces/default/pods/nginx-707a92bf-ee28-4961-9a51-e580525caf87',
              'uid': 'd9506caf-6d82-462f-8a72-874ede31186f'},
 'spec': {'act

In [18]:
# 创建deployment
nginx_deploy = {
    'apiVersion': 'apps/v1',
    'kind': 'Deployment',
    'metadata': {
        'name': 'nginx-deployment-python',
    },
    'spec': {
        'selector': {
            "matchLabels": {
                "app": "nginx"
            }
        },
        "replicas": 3,
        "template": {
            "metadata": {
                "labels": {
                    "app": "nginx"
                }
            },
            'spec': {
                'containers': [
                    {
                        'name': 'nginx',
                        'image': 'nginx:1.7.9',
                        "ports": [
                            {
                                "containerPort": 80
                            }
                        ]
                    }
                ]
            }
        }
    }
}
apps_v1.create_namespaced_deployment(namespace="default", body=nginx_deploy)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2023, 4, 11, 7, 59, 28, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'labels': None,
              'managed_fields': None,
              'name': 'nginx-deployment-python',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '45133',
              'self_link': '/apis/apps/v1/namespaces/default/deployments/nginx-deployment-python',
              'uid': '7a971e9c-d596-455b-8669-2728f45ee245'},
 'spec': {'min_ready_seconds': None,
          'paused': None,
          'progress_deadline_seconds': 600,
          'replicas': 3,
          'revision_history_limit': 10,
          'selector': {'match_expressions': None,
  

In [19]:
# 删除Deploy
apps_v1.delete_namespaced_deployment(namespace="default", name="nginx-deployment-python")

{'api_version': 'v1',
 'code': None,
 'details': {'causes': None,
             'group': 'apps',
             'kind': 'deployments',
             'name': 'nginx-deployment-python',
             'retry_after_seconds': None,
             'uid': '7a971e9c-d596-455b-8669-2728f45ee245'},
 'kind': 'Status',
 'message': None,
 'metadata': {'_continue': None,
              'remaining_item_count': None,
              'resource_version': None,
              'self_link': None},
 'reason': None,
 'status': 'Success'}

In [ ]:
# 创建一个CustomObjectsApi实例
# 定义CRD的组、版本和复数名称
group = "stable.example.com"
version = "v1"
plural = "crontabs"

# 定义CRD对象的元数据
metadata = client.V1ObjectMeta(name="my-new-cron-object")

# 定义CRD对象的规格
spec = {
    "cronSpec": "* * * * */5",
    "image": "my-awesome-cron-image"
}

# 创建CRD对象
crd_object = {
    "apiVersion": f"{group}/{version}",
    "kind": "CronTab",
    "metadata": metadata,
    "spec": spec
}

# 在"default"命名空间中创建CRD对象
custom_api.create_namespaced_custom_object(
    group=group,
    version=version,
    namespace="default",
    plural=plural,
    body=crd_object
)